In [1]:
import pandas as pd
import re
import os
from geopy.distance import distance as d

In [2]:
stop_times = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "stop_times.txt", delimiter=",")
stops = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs" + os.sep +  "stops.txt", delimiter=",")
routes = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "routes.txt", delimiter=",")
trips = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "trips.txt", delimiter=",")
calendar = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "calendar.txt", delimiter=",")

In [3]:
stop_times["arrival_time"] = pd.to_timedelta(stop_times["arrival_time"])

In [4]:
departure = "00:31:41"
destination =  (53.3404818, -6.2585706) #"Dawson Street, stop 793"
origin =  (53.3094124, -6.218878399999999) #"UCD Belfield, stop 768"
day = "Saturday"
day = day.lower()
bus_route = "39A"
bus_route = bus_route.lower()

In [11]:
def get_route(departure, olat, olng, dlat, dlng, day, bus_route):
    day = day.lower()
    bus_route = bus_route.lower()
    
    def trips_day(day):
        """Get all trips for that day"""
        days = list(calendar[calendar[day] == 1]["service_id"].values)
        day_trips = set(trips[trips["service_id"].isin(days)]["trip_id"].values)
        return day_trips
    
    def trips_route(bus_route):
        """Get all the trips for that route"""
        route_trips = set(trips[trips["route_id"].isin(list(routes[routes["route_short_name"] == bus_route]["route_id"].values))][
                "trip_id"].values)
        return route_trips
    
    def best_stop(trips, lat, lon):
        """Get the best corresponding stop for a list of trips"""
        possible_stop_ids = list(stop_times[stop_times["trip_id"].isin(list(trips))]["stop_id"].drop_duplicates())
        test = stops[stops["stop_id"].isin(possible_stop_ids)][["stop_name", "stop_id","stop_lat", "stop_lon"]]
        test["lat"] = [lat] * len(test)
        test["lon"] = [lon] * len(test)
        distances = []
        for index, row in test.iterrows():
            distance = abs(d((row["stop_lat"],row["stop_lon"]),(row["lat"], row["lon"])).km)
            distances.append(distance)
        test["distances"] = distances
        test = test.sort_values(by="distances")
        print(test[:1]["stop_name"].values[0])
        return test[:1]["stop_name"].values[0], test[:1]["stop_id"].values[0]
    
    def trips_with_origin_destination_day_route(trips, o_id, d_id):
        start = set(stop_times[stop_times["stop_id"] == o_id]["trip_id"].values)
        end = set(stop_times[stop_times["stop_id"] == d_id]["trip_id"].values)
        trips = start.intersection(end)
        trips = trips.intersection(trips)
        return trips
    
    def trips_correct_dir(trips, o_id, d_id):
        I, O = None, None
        for trip in r:
            if re.search(".I$", trip):
                I = trip
                break
        for trip in r:
            if re.search(".O$", trip):
                O = trip
                break
                
        correct_direction = []
        if O != None and ((stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == o_id)]["stop_sequence"].values[
                    0]) < (stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == d_id)][
                            "stop_sequence"].values[0])):
            correct_direction = [trip for trip in r if re.search(".O$", trip)]
        elif I != None and (
                (stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == o_id)]["stop_sequence"].values[
                    0]) < (                        stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == d_id)][
                            "stop_sequence"].values[0])):
            correct_direction = [trip for trip in r if re.search(".I$", trip)]    
        return set(correct_direction)
    
    def best_trip_for_time(r, departure, stop_id):
        final = stop_times[(stop_times["trip_id"].isin(r)) & (stop_times["stop_id"] == stop_id)][
            ["trip_id", "arrival_time"]]
        final["arrival_time"] = abs(final["arrival_time"] - pd.to_timedelta(departure))
        final = final.sort_values(by="arrival_time")
        #print(final)
        tripid = final.iloc[0]["trip_id"]   
        return tripid
    
    def station_to_station(tripid, o_id, d_id):
        seq = stop_times[stop_times["trip_id"] == tripid][["stop_id", "stop_sequence"]].sort_values(by="stop_sequence")
        rstart = seq[seq["stop_id"] == o_id].index[0]
        rfin = seq[seq["stop_id"] == d_id].index[0]
        test = seq.loc[rstart:rfin]
        final = test.merge(stops, how='left', on='stop_id')
        flist = list(final["stop_name"].values)
        flist = [stop.split(" stop ")[1] for stop in flist]
        final_routes = []
        for i in range(len(flist) - 1):
            final_routes.append(flist[i] + "_" + flist[i+1])
        final_routes
        return final_routes    
    
    r = trips_day(day).intersection(trips_route(bus_route)) # get intersection of trips for that route and day
    o_name, o_id = best_stop(r, olat, olng)
    d_name, d_id = best_stop(r, dlat, dlng)
    r = trips_with_origin_destination_day_route(r, o_id, d_id)
    r = trips_correct_dir(r, o_id, d_id)
    tripid = best_trip_for_time(r, departure, o_id)
    station_route = station_to_station(tripid, o_id, d_id)
    return station_route
    

In [12]:
get_route(departure, origin[0], origin[1], destination[0], destination[1], day, bus_route)

UCD Belfield, stop 768
Dawson Street, stop 793


['768_769',
 '769_770',
 '770_771',
 '771_772',
 '772_773',
 '773_774',
 '774_775',
 '775_776',
 '776_777',
 '777_779',
 '779_780',
 '780_781',
 '781_782',
 '782_783',
 '783_784',
 '784_785',
 '785_786',
 '786_793']

In [ ]:
get_route(departure, origin, destination, day, bus_route)

In [ ]:
flist = ['404',
 '747',
 '748',
 '749',
 '750',
 '751',
 '752',
 '753',
 '754',
 '755',
 '756',
 '757',
 '758',
 '759',
 '760',
 '761',
 '762',
 '763',
 '767']

In [ ]:
len(flist)

In [ ]:
final_routes = []
for i in range(len(flist) - 1):
    final_routes.append(flist[i] + "_" + flist[i+1])
final_routes

In [ ]:
test

In [ ]:
days = list(calendar[calendar[day] == 1]["service_id"].values)

In [ ]:
day_trips = set(trips[trips["service_id"].isin(days)]["trip_id"].values)

In [ ]:
route_trips = set(trips[trips["route_id"].isin(list(routes[routes["route_short_name"] == bus_route]["route_id"].values))]["trip_id"].values)

In [ ]:
start_trips = stops[stops["stop_name"] == origin]["stop_id"].values[0]
end_trips = stops[stops["stop_name"] == destination]["stop_id"].values[0]

In [ ]:
start_trips

In [ ]:
start  = set(stop_times[stop_times["stop_id"] == start_trips]["trip_id"].values)
end  = set(stop_times[stop_times["stop_id"] == end_trips]["trip_id"].values)

In [ ]:
end

In [ ]:
for trip in end:
    if "39A" in trip:
        print(trip)

In [ ]:
for trip in start:
    if "39A" in trip:
        print(trip)

In [ ]:
r = start.intersection(end)
r = r.intersection(route_trips)
r = r.intersection(day_trips)

In [ ]:
r

In [ ]:
for trip in r:
    if "39A" in trip:
        print(trip)

In [ ]:
I = None
O = None

for trip in r:
    if re.search(".I$", trip):
        I = trip
        break
        
for trip in r:
    if re.search(".O$", trip):
        O = trip
        break
print(O)
print(I)

In [ ]:
if O != None and ((stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == start)]["stop_sequence"].values[0]) < (stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == end)]["stop_sequence"].values[0])):
    correct_direction = [trip for trip in r if re.search(".O$", trip)]
elif I != None and ((stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == start)]["stop_sequence"].values[0]) < (stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == end)]["stop_sequence"].values[0])):
    correct_direction = [trip for trip in r if re.search(".I$", trip)]

In [ ]:
final = stop_times[(stop_times["trip_id"].isin(correct_direction)) & (stop_times["stop_id"] == start)][["trip_id","arrival_time"]]
final["arrival_time"] = abs(final["arrival_time"] - pd.to_timedelta(departure))
final = final.sort_values(by="arrival_time")

In [ ]:
final

In [ ]:
tripid = final.iloc[0]["trip_id"]

In [ ]:
seq = stop_times[stop_times["trip_id"] == tripid][["stop_id", "stop_sequence"]].sort_values(by="stop_sequence")

In [ ]:
rstart = seq[seq["stop_id"] == start].index[0]

In [ ]:
rfin = seq[seq["stop_id"] == end].index[0]

In [ ]:
test = seq.loc[rstart:rfin]

In [ ]:
final = test.merge(stops, how='left', on='stop_id')

In [ ]:
flist = list(final["stop_name"].values)
flist = [stop.split(" stop ")[1] for stop in flist]
flist

In [ ]:
from datetime import datetime

In [ ]:
datetime.fromtimestamp(1.62639e+18/ 1e9).strftime("%H")

In [ ]:
"00:14:40".split(":")[1]

In [13]:
t1 = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0, 'Friday': 0, 'Saturday': 1, 'Sunday': 0, 'timetabledtimes': 25.51075757972846, 'distance': -1.0302552736568087, 'rain': 0.0, 'temp': 13.550000000000011, 'rhum': 94.0, 'msl': 1025.0, 'holiday': 0, 'hour_0': 0, 'hour_1': 0, 'hour_5': 0, 'hour_6': 0, 'hour_7': 0, 'hour_8': 0, 'hour_9': 0, 'hour_10': 0, 'hour_11': 0, 'hour_12': 0, 'hour_13': 0, 'hour_15': 0, 'hour_16': 0, 'hour_17': 1, 'hour_18': 0, 'hour_19': 0, 'hour_20': 0, 'hour_21': 0, 'hour_22': 0, 'hour_23': 0}

In [20]:
len(t1)

34

In [18]:
t2 = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','timetabledtimes','distances','rain','temp','rhum','msl','holiday','hour_0','hour_1','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23']

In [22]:
for t in t2:
    if t not in t1.keys():
        print(t)

distances
hour_14


In [19]:
len(t2)

35

In [17]:
t2 = """Monday
Tuesday
Wednesday
Thursday
Friday
Saturday
Sunday
timetabledtimes
distances
rain
temp
rhum
msl
holiday
hour_0
hour_1
hour_5
hour_6
hour_7
hour_8
hour_9
hour_10
hour_11
hour_12
hour_13
hour_14
hour_15
hour_16
hour_17
hour_18
hour_19
hour_20
hour_21
hour_22
hour_23"""
t2.replace("\n", "','")

"Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','timetabledtimes','distances','rain','temp','rhum','msl','holiday','hour_0','hour_1','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23"

In [14]:
len(t1)

34